# Extract price datas from TradingView using the tvdatafeed package and Save results on google sheet 

### Objetive this work: 
   - Extract data from TradingView using package Tv Data Feed:
   [source code](https://github.com/StreamAlpha/tvdatafeed);
   - Executing some rule to filter data; 
   - Saving the results on a sheet in google sheet. 

The API of TradingView are avaliable on: https://br.tradingview.com/rest-api-spec/

## Instructions 

- install package tv datafeed - pip install --upgrade --no-cache-dir git+https://github.com/StreamAlpha/tvdatafeed.git  
- install gspread - pip install gspread
- install gsread_dataframe - pip install gspread_dataframe
- install pydrive - pip install pydrive
- install mplfinance - pip install mplfinance

The code source about this package are present in: https://github.com/StreamAlpha/tvdatafeed.  
To some examples: https://github.com/StreamAlpha/tvdatafeed/blob/main/tv.ipynb.  

## Import Packages and Load account 

In [3]:
import pandas as pd
import gspread

from datetime import date
from tvDatafeed import TvDatafeed, Interval
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from tqdm import tqdm

In [4]:
#First to configure and acess your account on TradingView: 
username = 'login_tradingview'
password = 'password_tradingview'

#tv = TvDatafeed(username, password)
tv = TvDatafeed()

you are using nologin method, data you access may be limited


## Load datas

In [5]:
stocks = ['PETR4', 'ITUB3', 'ITSA4', 'BBSE3']

In [8]:
data = pd.DataFrame()
for stock in tqdm(stocks):
    data = pd.concat([data, 
                        tv.get_hist(symbol = stock, exchange='BMFBOVESPA', interval=Interval.in_15_minute, n_bars = 100)])

100%|█████████████████████████████████████████████| 4/4 [00:03<00:00,  1.05it/s]


In [9]:
data.head()

,symbol,open,high,low,close,volume
datetime,,,,,,
2023-04-03 16:00:00,BMFBOVESPA:PETR4,24.38,24.44,24.36,24.42,2150100.0
2023-04-03 16:15:00,BMFBOVESPA:PETR4,24.42,24.49,24.42,24.46,2936100.0
2023-04-03 16:30:00,BMFBOVESPA:PETR4,24.46,24.47,24.35,24.43,3377000.0
2023-04-03 16:45:00,BMFBOVESPA:PETR4,24.43,24.50,24.41,24.48,2790900.0
2023-04-03 17:00:00,BMFBOVESPA:PETR4,24.49,24.49,24.49,24.49,3830800.0


In [10]:
data.shape

(400, 6)

## Processing data

The objetive are to extract just data with the price open is equal low.

In [11]:
tolerance = 0.0 # tolerance to open is higer than low price
cond = (data.open >= data.low) & (data.open <= data.low*(1+tolerance))

In [12]:
select_data = data[cond].reset_index()
select_data.shape

(79, 7)

In [13]:
select_data.set_index('symbol')

,datetime,open,high,low,close,volume
symbol,,,,,,
BMFBOVESPA:PETR4,2023-04-03 16:15:00,24.42,24.49,24.42,24.46,2936100.0
BMFBOVESPA:PETR4,2023-04-03 17:00:00,24.49,24.49,24.49,24.49,3830800.0
BMFBOVESPA:PETR4,2023-04-04 17:00:00,24.27,24.27,24.27,24.27,5336800.0
BMFBOVESPA:PETR4,2023-04-05 14:00:00,23.96,24.05,23.96,23.98,1042900.0
BMFBOVESPA:PETR4,2023-04-05 15:00:00,24.25,24.39,24.25,24.32,2857600.0
...,...,...,...,...,...,...
BMFBOVESPA:BBSE3,2023-04-05 17:45:00,32.70,32.73,32.70,32.72,800.0
BMFBOVESPA:BBSE3,2023-04-06 10:00:00,32.47,33.47,32.47,33.45,550800.0
BMFBOVESPA:BBSE3,2023-04-06 11:45:00,33.27,33.78,33.27,33.74,620100.0


In [14]:
select_data.datetime = select_data.datetime.astype('str')

## Save results on GoogleSheet

**Criar uma função para encapsular essa função também**

In [150]:
# Configure token and sheet id
json_file = 'your_file_json_toconnectgooglecloud'
sheet_id = 'id_of_google_sheet'

In [151]:
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(json_file, scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# open a google sheet
gs = gc.open_by_key(sheet_id)

# select a work sheet from its name
worksheet1 = gs.worksheet('Página1')

In [149]:
set_with_dataframe(worksheet=worksheet1, dataframe=select_data.sample(20), include_index=False,
                   include_column_header=True, resize=True)